# <center> **Consolidación de bases Población objetivo** 

In [41]:
import pandas as pd
import numpy as np

Para pegar las latitudes y longitudes, primero se agrupó la base de inegi por entihdad y municipio, por la clave de localidad mínima, ya que existen dos localidades en donde no existe la CVE con valor de 1. 

In [42]:
ineg = pd.read_excel('../datasets/AGEEML_2023311323291.xlsx') 
inegi_b1 = ineg.groupby(['CVE_ENT','CVE_MUN']).agg({'CVE_LOC':np.min}).reset_index()
inegi =inegi_b1.merge(ineg[['CVE_ENT','CVE_MUN','CVE_LOC', 'LAT_DECIMAL', 'LON_DECIMAL']], on=['CVE_ENT','CVE_MUN', 'CVE_LOC'], how='left')
inegi = inegi.drop('CVE_LOC', axis=1)
inegi.head()

,CVE_ENT,CVE_MUN,LAT_DECIMAL,LON_DECIMAL
0,1,1,21.88,-102.30
1,1,2,22.24,-102.09
2,1,3,21.85,-102.72
3,1,4,22.37,-102.30
4,1,5,21.96,-102.34


Teniendo la base de INEGI a nivel municipio armada, procedemos a leer la base de productores armar una sola base y, finalmente, pero no menos importante, pegar la latitud y longitud.

In [43]:
cols = ['Anio', 'CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'TotalCosechada',
       'TotalProductores', 'LAT_DECIMAL', 'LON_DECIMAL', 'GM', 'IM',
       'Escenario1', 'Escenario2']
arroz = pd.read_excel('../datasets/base_escenarios_arroz.xlsx')
arroz.columns = cols 
arroz['Producto'] = 'Arroz'
maiz = pd.read_excel('../datasets/base_escenarios_maiz.xlsx')
maiz.columns = cols 
maiz['Producto'] = 'Maíz'
trigo = pd.read_excel('../datasets/base_escenarios_trigo.xlsx')  
trigo.columns = cols 
trigo['Producto'] = 'Trigo'

# concatenación d ebases
base = pd.concat([arroz, maiz, trigo], axis=0)
#base['Anio'] = base['Anio'].astype('int')
base = base.drop(columns=['LAT_DECIMAL', 'LON_DECIMAL'])
base = base[['Anio', 'Producto', 'CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 
                    'TotalCosechada','TotalProductores', 'GM', 'IM', 'Escenario1', 'Escenario2']]
base.head()

,Anio,Producto,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,TotalCosechada,TotalProductores,GM,IM,Escenario1,Escenario2
0,2021,Arroz,7,Chiapas,32,Escuintla,80.50,21,Alto,0.823361,198200052.4,202374994.0
1,2021,Arroz,7,Chiapas,1,Acacoyagua,14.70,4,Medio,0.834460,13113504.4,201493102.0
2,2021,Arroz,6,Colima,9,Tecomán,383.35,19,Muy bajo,0.895601,NaN,201333859.6
3,2021,Arroz,7,Chiapas,3,Acapetahua,12.10,3,Alto,0.816596,198540936.4,189321829.6
4,2021,Arroz,4,Campeche,11,Candelaria,40.00,4,Alto,0.790978,199263586.0,189192208.0


Finalemnet unimos la base de productores con la de INEGI

In [44]:
productores = base.merge(inegi, on=['CVE_ENT','CVE_MUN'], how='left')
productores.head()

,Anio,Producto,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,TotalCosechada,TotalProductores,GM,IM,Escenario1,Escenario2,LAT_DECIMAL,LON_DECIMAL
0,2021,Arroz,7,Chiapas,32,Escuintla,80.50,21,Alto,0.823361,198200052.4,202374994.0,15.32,-92.66
1,2021,Arroz,7,Chiapas,1,Acacoyagua,14.70,4,Medio,0.834460,13113504.4,201493102.0,15.34,-92.67
2,2021,Arroz,6,Colima,9,Tecomán,383.35,19,Muy bajo,0.895601,NaN,201333859.6,18.91,-103.87
3,2021,Arroz,7,Chiapas,3,Acapetahua,12.10,3,Alto,0.816596,198540936.4,189321829.6,15.28,-92.69
4,2021,Arroz,4,Campeche,11,Candelaria,40.00,4,Alto,0.790978,199263586.0,189192208.0,18.18,-91.05


In [45]:
productores.isna().sum()

Anio                  0
Producto              0
CVE_ENT               0
NOM_ENT               0
CVE_MUN               0
NOM_MUN               0
TotalCosechada        0
TotalProductores      0
GM                    0
IM                    0
Escenario1          373
Escenario2          448
LAT_DECIMAL           0
LON_DECIMAL           0
dtype: int64

In [46]:
summary = productores.groupby(['Anio','Producto','CVE_ENT','CVE_MUN']).size().reset_index()
summary.to_excel('summary.xlsx')

In [47]:
productores.to_excel('TotalProductores.xlsx', index=False)
